In [5]:
from typing import List, Dict
from decimal import Decimal
from datetime import datetime


class Ride:
    def __init__(self, arr: List[str]):
        self.vendor_id = int(arr[0])
        self.tpep_pickup_datetime = datetime.strptime(arr[1], "%Y-%m-%d %H:%M:%S"),
        self.tpep_dropoff_datetime = datetime.strptime(arr[2], "%Y-%m-%d %H:%M:%S"),
        self.passenger_count = int(arr[3])
        self.trip_distance = Decimal(arr[4])
        self.rate_code_id = int(arr[5])
        self.store_and_fwd_flag = arr[6]
        self.pu_location_id = int(arr[7])
        self.do_location_id = int(arr[8])
        self.payment_type = arr[9]
        self.fare_amount = Decimal(arr[10])
        self.extra = Decimal(arr[11])
        self.mta_tax = Decimal(arr[12])
        self.tip_amount = Decimal(arr[13])
        self.tolls_amount = Decimal(arr[14])
        self.improvement_surcharge = Decimal(arr[15])
        self.total_amount = Decimal(arr[16])
        self.congestion_surcharge = Decimal(arr[17])

    @classmethod
    def from_dict(cls, d: Dict):
        return cls(arr=[
            d['vendor_id'],
            d['tpep_pickup_datetime'][0],
            d['tpep_dropoff_datetime'][0],
            d['passenger_count'],
            d['trip_distance'],
            d['rate_code_id'],
            d['store_and_fwd_flag'],
            d['pu_location_id'],
            d['do_location_id'],
            d['payment_type'],
            d['fare_amount'],
            d['extra'],
            d['mta_tax'],
            d['tip_amount'],
            d['tolls_amount'],
            d['improvement_surcharge'],
            d['total_amount'],
            d['congestion_surcharge'],
        ]
        )

    def __repr__(self):
        return f'{self.__class__.__name__}: {self.__dict__}'


def ride_to_dict(ride: Ride, ctx):
    return ride.__dict__

In [6]:
import csv
import os
from typing import List
# from ride import Ride

import os

CURRENT_FILE_PATH = os.getcwd()
INPUT_DATA_PATH = os.path.join(CURRENT_FILE_PATH, 'resources', 'data', 'rides.csv')


def read_rides(resource_path: str = INPUT_DATA_PATH) -> List[Ride]:
    rides = []
    with open(resource_path, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)  # skip the header row
        for row in reader:
            rides.append(Ride(arr=row))
    return rides

In [11]:
rides = []
with open(INPUT_DATA_PATH, 'r') as f:
    reader = csv.reader(f)
    header = next(reader)  # skip the header row
    for row in reader:
        rides.append(row)

In [20]:
rides = []
with open(INPUT_DATA_PATH, 'r') as f:
    reader = csv.reader(f)
    header = next(reader)  # skip the header row
    for row in reader:
        row_0 = row
        break

In [27]:
row_0[0]

'1'

In [28]:
datetime.strptime(row_0[1], "%Y-%m-%d %H:%M:%S")

datetime.datetime(2020, 7, 1, 0, 25, 32)